# Efficient Yelp API Calls

Craig Gossen

## Setting up API 

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open('/Users/craig/.secret/yelp_api.json') as f: 
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters 
LOCATION = 'Chicago, IL'
TERM = 'Pizza'

In [5]:
# Specifying JSON_FILE filename (can include a folder) #This sets the retrieved data into the DATA folder within the Jupyter notebook
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Chi_pizza.json"
JSON_FILE

'Data/results_in_progress_Chi_pizza.json'

In [8]:
#Install tqdm (only need to run once)
!pip install tqdm

## Create JSON File

In [9]:
#Function to create JSON file and check if existing file exists

def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f) 

## Interact with API

In [12]:
# Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
print(f'The total number of results is: {total_results}')

## How many did we get the details for?
results_per_page = len(results['businesses'])
print(f'We can retrieve {results_per_page} results at a time (per page)')

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
print(f'The number of pages is: {n_pages}')

[!] Data/results_in_progress_Chi_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_Chi_pizza.json not found. Saving empty list to new file.
- 0 previous results found.
The total number of results is: 7700
We can retrieve 20 results at a time (per page)
The number of pages is: 385


The total number of results is 7700, which is >1000, and will knowingly stop the API requests. 

In [13]:
#Loop to iterate and combine the results per page up to 1000 api calls

for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.1)

  0%|          | 0/385 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## Convert .json file to dataframe

In [14]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/FXGmfo...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,6982,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[pickup, delivery]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,10366.447105
1,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3375,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88513231411071, 'longitude': -...",[delivery],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13126161200,(312) 616-1200,10689.526750
2,DXwSYgiXqIVNdO9dazel6w,pequods-pizzeria-chicago,Pequod's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/8QJUNb...,False,https://www.yelp.com/biz/pequods-pizzeria-chic...,7369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.92187, 'longitude': -87.664486}","[pickup, delivery]",$$,"{'address1': '2207 N Clybourn Ave', 'address2'...",+17733271512,(773) 327-1512,11423.025038
3,cxAkuwv7p4gm4Coixw7h9A,lou-malnatis-pizzeria-chicago-9,Lou Malnati's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/vZVADY...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,2954,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.871566, 'longitude': -87.627275}","[delivery, pickup]",$$,"{'address1': '805 S State St', 'address2': '',...",+13127861000,(312) 786-1000,9676.008122
4,hxq4TNNazA-sjqFvVgx_Pw,bongiornos-italian-deli-and-pizzeria-chicago,Bongiorno's Italian Deli & Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/A2VP2G...,False,https://www.yelp.com/biz/bongiornos-italian-de...,761,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.889903, 'longitude': -87.625788}","[pickup, delivery]",$$,"{'address1': '405 N Wabash Ave', 'address2': '...",+13127551255,(312) 755-1255,10855.422434


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,OePx82TBE3PuHJH-7OfLHQ,antonios-pizza-factory-geneva,Antonio's Pizza Factory,https://s3-media4.fl.yelpcdn.com/bphoto/vEPxj7...,False,https://www.yelp.com/biz/antonios-pizza-factor...,47,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.8827095031738, 'longitude': -8...",[delivery],$$,"{'address1': '500 Lark St', 'address2': '', 'a...",+16302326666,(630) 232-6666,50978.379399
996,kmJ6Todwg36wLxv6UZYMyA,chicago-pizza-authority-hoffman-estates,Chicago Pizza Authority,https://s3-media3.fl.yelpcdn.com/bphoto/5sWngZ...,False,https://www.yelp.com/biz/chicago-pizza-authori...,24,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 42.09277, 'longitude': -88.12274}","[delivery, pickup]",NaN,"{'address1': '1726 W Algonquin Rd', 'address2'...",+18474858223,(847) 485-8223,43409.584245
997,fESy8pfywnwDqJzKpn5GZw,sarpinos-pizzeria-chicago-452,Sarpino's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/390GS4...,False,https://www.yelp.com/biz/sarpinos-pizzeria-chi...,213,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 41.9613, 'longitude': -87.66648}","[delivery, pickup]",$$,"{'address1': '1435 W Montrose St', 'address2':...",+17732811800,(773) 281-1800,15346.641319
998,fDpryalBDQhHoll8UKRaXw,way-out-chicago,Way Out,https://s3-media2.fl.yelpcdn.com/bphoto/puRdxB...,False,https://www.yelp.com/biz/way-out-chicago?adjus...,22,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,"{'latitude': 41.91716335791234, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '3213 W Armitage Ave', 'address2'...",+17737998750,(773) 799-8750,9670.088104
999,q9AUsrYTakTMIHJkKA_EIQ,exchequer-restaurant-and-pub-chicago,Exchequer Restaurant & Pub,https://s3-media2.fl.yelpcdn.com/bphoto/IWNK1y...,False,https://www.yelp.com/biz/exchequer-restaurant-...,630,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.8787384660152, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '226 S Wabash Ave', 'address2': '...",+13129395633,(312) 939-5633,10131.153858


In [15]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

6

In [17]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [18]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Chi_pizza.csv.gz', compression='gzip',index=False)